In [ ]:
!wget https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
!unzip full_grams_cbow_300_twitter.zip

--2022-03-14 20:17:22--  https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip [following]
--2022-03-14 20:17:23--  https://ia802803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving ia802803.us.archive.org (ia802803.us.archive.org)... 207.241.232.113
Connecting to ia802803.us.archive.org (ia802803.us.archive.org)|207.241.232.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325529808 (3.1G) [application/zip]
Saving to: ‘full_grams_cbow_300_twitter.zip’

                 fu  20%[===>                ] 648.38M   595KB/s    eta 78m 27s

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('bmh')
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100000)
# import Data_Preprocessing as dp
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
from tensorflow.keras.models import load_model
from xgboost import XGBClassifier


2022-03-14 20:12:10.774128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 20:12:10.774162: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
df=pd.read_csv('final_data_cleaned.csv')
df

,Unnamed: 0,dialect,content
0,0,IQ,لكن بالنهاية ينتفض يغير
1,1,IQ,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب
2,2,IQ,مبين كلامه خليجي
3,3,IQ,يسلملي مرورك وروحك الحلوه
4,4,IQ,وين هل الغيبه اخ محمد
...,...,...,...
458192,458192,BH,مبسوطين باسطانا
458193,458193,BH,واله ماينده ابش يختي
458194,458194,BH,شو عملنا لك حنا تهربي منا مساكين ليش بتعملي هيك فينا
458195,458195,BH,اله يبارك وبالعافيه


In [17]:
y = df['dialect'].astype(str)
X= df['content'].astype(str)

In [18]:
X_train , X_test , Y_train , Y_test = train_test_split(X , y,test_size=0.2 ,stratify=df['dialect'], random_state= 0)

# LSTM Model

In [19]:
t = Tokenizer()
t.fit_on_texts(X_train)

In [20]:
pickle.dump(t, open('tokenizer.pkl', 'wb'))

In [21]:
t=pickle.load(open('tokenizer.pkl', 'rb'))

In [22]:
vocab_size = len(t.word_index) + 1
vocab_size

425968

In [23]:
# integer encode the documents
encoded_X_train = t.texts_to_sequences(X_train)
encoded_X_test = t.texts_to_sequences(X_test)

In [24]:
max_len=0
for i in X_train:
    max_len=max(len(i),max_len)
print(max_len)

280


In [25]:
# pad documents to a max length of 4 words
padded_X_train = pad_sequences(encoded_X_train, maxlen=max_len, padding='post')
padded_X_test = pad_sequences(encoded_X_test, maxlen=max_len, padding='post')

In [26]:
len(padded_X_train)

366557

In [ ]:
# load the whole embedding into memory
w2v_embeddings_index={}
TOTAL_EMBEDDING_DIM=300
embeddings_file="D:/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.mdl"
w2v_model =KeyedVectors.load(embeddings_file)
for word in w2v_model.wv.vocab:
    w2v_embeddings_index[word] = w2v_model[word]
print('Loaded %s word vectors.'% len(w2v_embeddings_index))

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, TOTAL_EMBEDDING_DIM))
for word, i in t.word_index.items():
    embedding_vector = w2v_embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Embedding Matrix shape:', embedding_matrix.shape)

In [ ]:
mapping={'AE': 16,'BH': 17,'DZ': 10,'EG': 11,'IQ': 0,'JO': 6,'KW': 13,'LB': 12,'LY': 1,'MA': 7,'OM': 14,'PL': 3,
         'QA': 2,'SA': 8,'SD': 15,'SY': 4,'TN': 5,'YE': 9}

In [ ]:
Y_train = Y_train.apply(lambda x: mapping[x])
Y_test = Y_test.apply(lambda x: mapping[x])

In [ ]:
Y_train=tf.keras.utils.to_categorical(Y_train, num_classes = 18)
Y_test=tf.keras.utils.to_categorical(Y_test, num_classes = 18)

In [ ]:
bilstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, TOTAL_EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_len, trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(18, activation='softmax'),
                             ])

In [ ]:
bilstm.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
bilstm.summary()

In [ ]:
history = bilstm.fit(padded_X_train,
                     Y_train,
                     epochs = 5,
                     verbose = 1)

In [ ]:
bilstm.save('lstm_model.h5')

In [ ]:
history.model.evaluate(padded_X_test, Y_test)

In [ ]:
lstm_model = load_model('lstm_model.h5')
tok = pickle.load(open('tokenizer.pkl', 'rb'))

In [ ]:
X_test_lstm = tok.texts_to_sequences(X_test)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=280, padding='post')

In [ ]:
Y_pred_lstm = lstm_model.predict(X_test_lstm)

In [ ]:
Y_pred_lstm1=[np.argmax(i) for i in Y_pred_lstm]

In [ ]:
for i in range(len(Y_pred_lstm1)):
    for k, v in mapping.items():
        if v == Y_pred_lstm1[i]:
            Y_pred_lstm1[i] = k

In [ ]:
Y_pred_lstm = pd.Series(Y_pred_lstm1)

In [ ]:
print(classification_report(Y_test , Y_pred_lstm ,target_names=df['dialect'].unique()))